<a href="https://colab.research.google.com/github/soohyoen/artificial-intelligence/blob/main/C_recommend_movies_class_1_ipynb%EC%9D%98_%EC%82%AC%EB%B3%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# install and import the libraries needed
# this script is heavily inspired by Wikidocs
!pip3 install scikit-learn
!pip3 install gdown
from typing import Tuple, List
import numpy as np
import pandas as pd
from scipy.spatial.distance import pdist
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity, euclidean_distances, manhattan_distances
import os
import gdown # for downloading the data stored in google drive

In [ ]:
# --- constants --- #
MOVIES_METADATA_CSV_URL = "https://drive.google.com/u/0/uc?id=15I-izNgUJqybrPLK3qlZhLMCuuNcaJhJ&export=download"
MOVIES_METADATA_CSV = "./movies_metadata.csv"  # save the data in the current directory
DOC_SIZE = 2000  # choose the size of the data here (너무 크면 시간이 오래 걸릴 수도 있음!)
TOP_N = 30

In [ ]:
# download the data
gdown.download(url=MOVIES_METADATA_CSV_URL, output=MOVIES_METADATA_CSV, quiet=False)

Downloading...
From: https://drive.google.com/u/0/uc?id=15I-izNgUJqybrPLK3qlZhLMCuuNcaJhJ&export=download
To: /content/movies_metadata.csv
34.4MB [00:01, 34.1MB/s]


'./movies_metadata.csv'

In [ ]:
# have a look at the data
movies_df = pd.read_csv(MOVIES_METADATA_CSV, low_memory=False)
movies_df.head(3)

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,popularity,poster_path,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.946943,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,"[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,17.015539,/vzmL6fP7aPKNKPRTFnZmiUfciyV.jpg,"[{'name': 'TriStar Pictures', 'id': 559}, {'na...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,11.7129,/6ksm1sjKMFLbO7UY2i6G1ju9SML.jpg,"[{'name': 'Warner Bros.', 'id': 6194}, {'name'...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0


In [ ]:
# resize the dataframe
movies_df = movies_df.head(DOC_SIZE)
# we don't want any null values. So replace them with an empty string.
movies_df['overview'] = movies_df['overview'].fillna('')

In [ ]:
### TODO 1 #### 
# use CountVectorizer & TfidfVectorizer to construct dtm & dtm_tfidf
count = CountVectorizer(binary=False) #binary x, tf
tfidf = TfidfVectorizer()
dtm: np.ndarray = count.fit_transform(movies_df['overview']).toarray()
dtm_tfidf: np.ndarray = tfidf.fit_transform(movies_df['overview']).toarray()  # from csr_matrix to numpy array.  # 이제는 이거 한줄로 끝내기!
###############
print(dtm)  # should be csr sparse matrix
print(dtm.shape)  # (num_docs, num_terms) 당연히.. 대부분은 0 이겠지! - 다시한번 sparsity를 확인할 수 있다.
print(dtm_tfidf)  # should be csr sparse matrix
print(dtm_tfidf.shape)  # (num_docs, num_terms) 당연히.. 대부분은 0 이겠지! - 다시한번 sparsity를 확인할 수 있다.

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
(2000, 14195)
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
(2000, 14195)


In [ ]:
# cosine distance -> this is vectorized...
sims_cosine = cosine_similarity(dtm)
sims_cosine_tfidf = cosine_similarity(dtm_tfidf)  # this may take a while... 
print(sims_cosine.shape)  # (num_doc, num_doc)
print(sims_cosine_tfidf.shape)  # (num_doc, num_doc)

(2000, 2000)
(2000, 2000)


In [ ]:
# Note that these are distances, now
dists_manhattan_tfidf = manhattan_distances(dtm_tfidf)
print(dists_manhattan_tfidf.shape)  # (num_doc, num_doc)

(2000, 2000)


In [ ]:
dists_euclidean_tfidf = euclidean_distances(dtm_tfidf)  # this may take a while... not efficient at all? cosine - it didn't take that much of a time. but manhattan... well that took some time.
print(dists_euclidean_tfidf.shape)  # (num_doc, num_doc)

(2000, 2000)


In [ ]:
# this is to retrive the titles
indices = pd.Series(data=movies_df.index, index=movies_df['title']).drop_duplicates()
indices.head(5)

title
Toy Story                      0
Jumanji                        1
Grumpier Old Men               2
Waiting to Exhale              3
Father of the Bride Part II    4
dtype: int64

In [ ]:
batman_idx = indices["Batman"]
batman_returns_idx = indices["Batman Returns"]
toy_story_idx = indices["Toy Story"]
print(sims_cosine_tfidf[batman_idx, batman_returns_idx])
print(sims_cosine_tfidf[batman_idx, toy_story_idx])


0.19155692273625355
0.01586093997388227


In [ ]:
print(type(indices))

<class 'pandas.core.series.Series'>


In [ ]:
def get_recommendations(title: str,
                        # dists X
                        sims: np.ndarray,
                        indices: pd.Series,
                        data: pd.DataFrame) -> List[Tuple[str, float]]:
    global TOP_N
    ### TODO 2 ###
    # 루프는 사용 X
    # use: np.argsort()[::-1] to recommend titles with scores
    title_idx = indices[title]
    sims_to_title = sims[title_idx, :] # (2000,)
    sims_to_title_sorted: List[float] = np.sort(sims_to_title)[::-1]
    indices_sorted = np.argsort(sims_to_title)[::-1]
    titles_sorted: List[str] = data['title'][indices_sorted]
    results: List[Tuple[str, float]] = [
        (title, score)
        for title, score in zip(titles_sorted, sims_to_title_sorted)
    ]
    ##############
    return results[:TOP_N]

In [ ]:
my_list = [1, 2, 3]
print(my_list[::-1])

[3, 2, 1]


In [ ]:
# compare the results of dtm with tfidf
# 이유는 다들 알고 계십니다! ㅎㅎ
print("#### dtm + cosine sim ####")
for idx, (title, score) in enumerate(get_recommendations("Batman", sims_cosine, indices, movies_df)):
  print(idx, title, score)
print("#### tfidf + cosine sim ####")
for idx, (title, score) in enumerate(get_recommendations("Batman", sims_cosine_tfidf, indices, movies_df)):
  print(idx, title, score)

#### dtm + cosine sim ####
0 Batman 1.0
1 Mary Shelley's Frankenstein 0.4526831422644854
2 The Leopard Son 0.4468418247369901
3 The Last Emperor 0.44633201988370275
4 The Confessional 0.4436069753671345
5 Sweet Nothing 0.43970404417590525
6 Conan the Barbarian 0.4375949744936837
7 The Children Are Watching Us 0.43193421279068006
8 The Search for One-eye Jimmy 0.43092875104518874
9 Stalingrad 0.42857142857142855
10 North Star 0.42839304295347314
11 I Can't Sleep 0.42766686606638943
12 The Rocketeer 0.42721034696687243
13 Days of Thunder 0.4269760205378603
14 The Remains of the Day 0.42525863589985735
15 My Life and Times With Antonin Artaud 0.4198925212700719
16 The Replacement Killers 0.4190262407031392
17 Ed Wood 0.4164633650362828
18 Commandments 0.41475753100312657
19 Return of the Jedi 0.4139469470433775
20 And the Band Played On 0.41226488477260886
21 Mad Dog Time 0.4114755998989118
22 Child's Play 2 0.41099746826339323
23 The Education of Little Tree 0.4103049699311091
24 The Big

In [ ]:
### TODO 3 ###
# use different similarity measures and compare the results
# note that dists != sims
print("#### tfidf + cosine sim ####")
for idx, (title, score) in enumerate(get_recommendations("Batman", sims_cosine_tfidf, indices, movies_df)):
  print(idx, title, score)
print("\n#### tfidf + euclidean dists ####")
for idx, (title, score) in enumerate(get_recommendations("Batman", -dists_euclidean_tfidf, indices, movies_df)):
  print(idx, title, score)
print("\n#### tfidf + manhattan dists ####")
for idx, (title, score) in enumerate(get_recommendations("Batman", -dists_manhattan_tfidf, indices, movies_df)):
  print(idx, title, score) 
##############   

#### tfidf + cosine sim ####
0 Batman 1.0000000000000002
1 Batman Returns 0.19155692273625355
2 Batman & Robin 0.17582959957961833
3 Batman Forever 0.14581071757552014
4 Cry, the Beloved Country 0.11109312245823069
5 B. Monkey 0.10544235266596627
6 The Shadow 0.10514679108660141
7 Sweet Nothing 0.10035634826245518
8 A Month by the Lake 0.10023026852720504
9 Chairman of the Board 0.09920814012584121
10 The War at Home 0.0988387365764903
11 Dune 0.09875401691372578
12 Mediterraneo 0.09803176049698262
13 Return of the Jedi 0.09217825522595026
14 The Pompatus of Love 0.09085969169749389
15 Cyclo 0.08667634812405352
16 Commandments 0.08651635230910235
17 Apocalypse Now 0.08609774357728836
18 Friday the 13th: A New Beginning 0.08605652456139677
19 Dangerous Game 0.08604236366845823
20 Highlander 0.08602706777821761
21 Metropolis 0.08398384733774258
22 Somebody Is Waiting 0.08258337613352316
23 M 0.08193657697542574
24 Mrs. Miniver 0.0811774410090339
25 The Apostle 0.08031383520515435
26 Love

# 다음의 문제에 답하세요.
> cosine >> euclidean & manhattan 인 이유는 무엇인가요?
- 이승환11:20
코사인 유사도 방법은 길이와 관련없이 벡터간의 각도만을 고려해서  (줄거리 문서 -> 단어의 빈도수가 제각기이기 때문에, 각도만을 고려하는 것이 적합).


> 일반적으로 고차원 벡터인 경우, manhattan이 euclidean보다 성능이 좋은 것으로 알려져 있습니다 (슬라이드). 하지만 이 문제에서는 euclidean > manhattan 입니다. 그 이유는 무엇인가요?
논문: "일반적" = 정규분포, 분포, 일반적으로 분포를 가정 (벡터 속의 성분). -> [100, 1 ,2]
힌트: 벡터를 얻은 방법은 규칙기반. tfidf. (벡터 속의 성분 -> keyword는 값이 높다). 
outlier (이상치). -> [100 <- 중요한 단어, 1 ,2]
유클리드 -> 가중치가 증폭 (제곱).
맨해튼 거리 -> 둔감하다.
차원의 저주 -> 고차원으로 갈수록 거리의 개념 옅어지는데 (모두가 다 커지니까).
일반적인 주장의 반례.

